This cell sets up the Google Colab environment to handle asynchronous programming and enables the display of rich HTML content.

1. **`import nest_asyncio`**: Imports the `nest_asyncio` library, which allows for the smooth handling of asynchronous tasks in environments like Google Colab, where nested event loops might otherwise cause issues.

2. **`nest_asyncio.apply()`**: Applies a patch that enables running asynchronous code without conflicts. This is particularly useful when working with `asyncio` in notebooks, allowing for proper execution of asynchronous tasks.

3. **`from IPython.display import display, HTML`**: Imports functions from IPython’s display module that allow for the rendering of rich media content (like HTML) directly within the notebook. This enables the output of formatted HTML, which can be used for interactive content or visually enriched outputs.

This setup ensures that you can effectively run and display asynchronous code alongside HTML content in the same notebook.

In [12]:
import nest_asyncio

nest_asyncio.apply()

from IPython.display import display, HTML

Ensure you have Ollama installed

In [16]:
!which ollama

/usr/local/bin/ollama


To install Ollama: https://ollama.com/download

Ensure you have Mistral in Ollama

In [15]:
!ollama list

NAME          	ID          	SIZE  	MODIFIED          
mistral:latest	f974a74358d6	4.1 GB	About an hour ago	
llama3:latest 	365c0bd3c000	4.7 GB	2 months ago     	


To install Mistral on Ollamarun the following cell

In [ ]:
!ollama pull mistral

In [3]:
%pip install ollama --quiet


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
import json
import ollama
import asyncio


# Simulates an API call to get flight times
# In a real application, this would fetch data from a live database or API
def get_flight_times(departure: str, arrival: str) -> str:
  flights = {
    'NYC-LAX': {'departure': '08:00 AM', 'arrival': '11:30 AM', 'duration': '5h 30m'},
    'LAX-NYC': {'departure': '02:00 PM', 'arrival': '10:30 PM', 'duration': '5h 30m'},
    'LHR-JFK': {'departure': '10:00 AM', 'arrival': '01:00 PM', 'duration': '8h 00m'},
    'JFK-LHR': {'departure': '09:00 PM', 'arrival': '09:00 AM', 'duration': '7h 00m'},
    'CDG-DXB': {'departure': '11:00 AM', 'arrival': '08:00 PM', 'duration': '6h 00m'},
    'DXB-CDG': {'departure': '03:00 AM', 'arrival': '07:30 AM', 'duration': '7h 30m'},
  }

  key = f'{departure}-{arrival}'.upper()
  return json.dumps(flights.get(key, {'error': 'Flight not found'}))


async def run(model: str):
  client = ollama.AsyncClient()
  # Initialize conversation with a user query
  messages = [{'role': 'user', 'content': 'What is the flight time from New York (NYC) to Los Angeles (LAX)?'}]

  # First API call: Send the query and function description to the model
  response = await client.chat(
    model=model,
    messages=messages,
    tools=[
      {
        'type': 'function',
        'function': {
          'name': 'get_flight_times',
          'description': 'Get the flight times between two cities',
          'parameters': {
            'type': 'object',
            'properties': {
              'departure': {
                'type': 'string',
                'description': 'The departure city (airport code)',
              },
              'arrival': {
                'type': 'string',
                'description': 'The arrival city (airport code)',
              },
            },
            'required': ['departure', 'arrival'],
          },
        },
      },
    ],
  )

  # Add the model's response to the conversation history
  messages.append(response['message'])

  # Check if the model decided to use the provided function
  if not response['message'].get('tool_calls'):
    print("The model didn't use the function. Its response was:")
    print(response['message']['content'])
    return

  # Process function calls made by the model
  if response['message'].get('tool_calls'):
    available_functions = {
      'get_flight_times': get_flight_times,
    }
    for tool in response['message']['tool_calls']:
      function_to_call = available_functions[tool['function']['name']]
      function_response = function_to_call(tool['function']['arguments']['departure'], tool['function']['arguments']['arrival'])
      # Add function response to the conversation
      messages.append(
        {
          'role': 'tool',
          'content': function_response,
        }
      )

  # Second API call: Get final response from the model
  final_response = await client.chat(model=model, messages=messages)
  print(final_response['message']['content'])


# Run the async function
asyncio.run(run('mistral'))

The flight time from New York (NYC) to Los Angeles (LAX) is approximately 5 hours and 30 minutes, with a departure time of 8:00 AM.
